In [18]:
import numpy as np
import tensorflow as tf
import os
import datetime

#Some hyperparameters
training_iters = 10000 
learning_rate = 0.0001 
batch_size = 16
image_size = 64
num_channels = 3
num_classes = 2
p = [0.50, 0.50]
#The Model Itself
def cnn_model(in_data):
    bn = tf.layers.batch_normalization(in_data)
    
    conv1 = tf.layers.conv2d(inputs=bn, filters=64, kernel_size= 5) 
    pool1 = tf.layers.MaxPooling2D(pool_size=3, strides=2)(conv1)
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm1')
    
    conv2 = tf.layers.conv2d(inputs=norm1, filters=64, kernel_size= 3) 
    pool2 = tf.layers.MaxPooling2D(pool_size=3, strides=2)(conv2)
    flat = tf.layers.Flatten()(pool2)
    
    fc1 = tf.layers.dense(flat, 384)
    fc2 = tf.layers.dense(fc1, 192)
    output = tf.layers.dense(fc2,num_classes)
    return output


#Helper function for parsing filenames into images
def parse_function(filename, label):
    with tf.name_scope("parse_function"):
        image_string = tf.read_file(filename)

        # Don't use tf.image.decode_image, or the output shape will be undefined
        image = tf.image.decode_jpeg(image_string)
        image = tf.image.random_crop(image, [image_size,image_size,num_channels])
        image = tf.image.convert_image_dtype(image, tf.float32)
        # We know that each image in this dataset is 64 x 64 pixels + has 3 color channels.
        #image.set_shape([image_size, image_size, num_channels])
        return image, label

#Helper function for artifically increasing variety of training data
def train_preprocess(image, label):
    with tf.name_scope("train_preprocess"):
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_jpeg_quality(image, 90, 100)
        image = tf.image.random_saturation(image, 0.75, 1.25)
        image = tf.image.random_contrast(image, 0.75, 1.25)
        image = tf.image.random_brightness(image, 1.25)
        image.set_shape([image_size, image_size, num_channels])
        return image, label

#Helper function for fixing class imbalance
def random_choice(p):
    choice = tf.multinomial(tf.log([p]), 1)
    return tf.cast(tf.squeeze(choice), tf.int64)
    
#Reset graph so tensorboard doesn't kill itself
tf.reset_default_graph()

#Make directory to write log data to
UID =  str(datetime.datetime.now()).replace(' ', '_').replace(':', '_').replace('.', '_')
logPath = "./logs/Waldo2" + UID
os.mkdir(logPath)

#Make paths to pull data from
notwaldoPath = os.path.join("Training_Images", "NotWaldo")
waldoPath = os.path.join("Training_Images", "Waldo")

#Collect filesnames and labels for samples
notwaldo_filenames = [os.path.join(notwaldoPath,filename) for filename in os.listdir(notwaldoPath)]
notwaldo_labels = [(1,0) for filename in os.listdir(notwaldoPath)]
waldo_filenames = [os.path.join(waldoPath,filename) for filename in os.listdir(waldoPath)]
waldo_labels = [(0,1) for filename in os.listdir(waldoPath)]

#Make two datasets : one with waldo one without
waldo_set = tf.data.Dataset.from_tensor_slices( (waldo_filenames, waldo_labels) )
waldo_set = waldo_set.shuffle(len(waldo_filenames)).repeat()
notwaldo_set = tf.data.Dataset.from_tensor_slices( (notwaldo_filenames, notwaldo_labels) )
notwaldo_set = notwaldo_set.shuffle( len(notwaldo_filenames) ).repeat()

#Interleave those two datasets together into the training dataset
datasets = [notwaldo_set, waldo_set]
choice_dataset = tf.data.Dataset.from_tensors([0])
choice_dataset = choice_dataset.map(lambda z: random_choice(p)).repeat()
dataset = tf.data.experimental.choose_from_datasets(datasets, choice_dataset)
dataset = dataset.map(parse_function, num_parallel_calls=4)
dataset = dataset.map(train_preprocess, num_parallel_calls=4)
dataset = dataset.repeat()
dataset = dataset.batch(batch_size)
dataset = dataset.prefetch(1)

#Make the iterator
iterator = tf.data.Iterator.from_structure(dataset.output_types, dataset.output_shapes)
next_element = iterator.get_next()
training_init_op = iterator.make_initializer(dataset)

#define all the learny things
logits = cnn_model(next_element[0])
loss = tf.reduce_sum(tf.nn.softmax_cross_entropy_with_logits_v2(labels = next_element[1], logits = logits))
optimizer = tf.train.AdamOptimizer().minimize(loss)
prediction = tf.argmax(logits, 1)
equality = tf.equal(prediction, tf.argmax(next_element[1],1))
accuracy = tf.reduce_mean(tf.cast(equality, tf.float32))

def train():
    epochs = training_iters
    with tf.Session() as sess:
        #Collect literally all the data
        writer = tf.summary.FileWriter(logPath)
        writer.add_graph(sess.graph)
        
        tf.summary.scalar('Softmax_Cross_Entropy_Loss', loss)
        tf.summary.scalar('accuracy', accuracy)
        tf.summary.histogram('Predictions', prediction)
        tf.summary.histogram("BN_gamma0", sess.graph.get_tensor_by_name('batch_normalization/gamma:0'))
        tf.summary.histogram("BN_beta0", sess.graph.get_tensor_by_name('batch_normalization/beta:0'))
        tf.summary.histogram("conv_kernel0", sess.graph.get_tensor_by_name('conv2d/kernel:0'))
        tf.summary.histogram("conv_bias0", sess.graph.get_tensor_by_name('conv2d/bias:0'))
        tf.summary.histogram("conv_kernel1", sess.graph.get_tensor_by_name('conv2d_1/kernel:0'))
        tf.summary.histogram("conv_bias1", sess.graph.get_tensor_by_name('conv2d_1/bias:0'))
        tf.summary.histogram("fc_kernel0", sess.graph.get_tensor_by_name('dense/kernel:0'))
        tf.summary.histogram("fc_bias0", sess.graph.get_tensor_by_name('dense/bias:0'))
        tf.summary.histogram("fc_kernel1", sess.graph.get_tensor_by_name('dense_1/kernel:0'))
        tf.summary.histogram("fc_bias1", sess.graph.get_tensor_by_name('dense_1/bias:0'))
        tf.summary.histogram("fc_kernel2", sess.graph.get_tensor_by_name('dense_2/kernel:0'))
        tf.summary.histogram("fc_bias2", sess.graph.get_tensor_by_name('dense_2/bias:0'))
        tf.summary.image("input_data", next_element[0])    
        sess.run(tf.global_variables_initializer())

        
        sess.run(training_init_op)
        print("Beginning Training")
        for i in range(epochs):
            l, _, acc = sess.run([loss, optimizer, accuracy])
            if i % 20 == 0:
                #print("Epoch: {}, loss: {:.3f}, training accuracy: {:.2f}%".format(i, l, acc * 100))
                merged = tf.summary.merge_all()
                s = sess.run(merged)
                writer.add_summary(s, i)
train()
print("Final Accuracy:, Testing Accuracy:")

Beginning Training
